In [1]:
import tensorflow as tf
print('Using Tensorflow '+tf.__version__)
assert tf.__version__>='1.0.0', ('You should use Tensorflow 1.0 or superior')
#import matplotlib
import matplotlib.patches as patches
import matplotlib.pyplot as plt
#matplotlib.use('Agg')
#%matplotlib inline
import sys
sys.path.append('../')
import os.path
import csv
import numpy as np

from src.argparse import argparse
from src.region_to_bbox import region_to_bbox
from src.pprint_params import pprint_params
from src.Tracker import Tracker
from src.crops import *
from src.siamese import import_siamese

Using Tensorflow 1.1.0


In [2]:
hp = {"z_lr":0}
evaluation = {"video": "vot2016_ball1"}
run = {"visualization":0,"debug":0}
hp,evaluation,run,env,design = argparse(hp, evaluation, run)
pprint_params((hp,evaluation,run,env,design),['tracker hyperparams','evaluation','run','environment','design'])


#### SETTINGS

## tracker hyperparams
hp(scale_min=0.2, window_influence=0.25, z_lr=0, scale_max=5, scale_step=1.04, scale_num=3, scale_penalty=0.97, response_up=8, scale_lr=0.59)

## evaluation
evaluation(start_frame=0, TRE_th_points=101, dataset=u'validation', video='vot2016_ball1', TRE_subseq=3, stop_on_failure=0)

## run
run(debug=0, visualization=0, gpus=[1])

## environment
env(root_pretrained=u'../pretrained', root_parameters=u'../parameters', root_dataset=u'../data')

## design
design(exemplar_sz=127, net_gray=u'', search_sz=255, join_method=u'xcorr', tot_stride=4, context=0.5, net=u'baseline-conv5_e55.mat', windowing=u'cosine_sum', score_sz=33)


In [3]:
video_folder = os.path.join(env.root_dataset, evaluation.dataset, evaluation.video)
gt_file = os.path.join(video_folder, 'groundtruth.txt')
gt = np.genfromtxt(gt_file, delimiter=',')
# bbox is in format <cx,cy,w,h>
init_bbox = region_to_bbox(gt[evaluation.start_frame])
frame_name_list = [f for f in os.listdir(video_folder) if f.endswith(".jpg")]
frame_name_list.sort()
assert len(gt) == len(frame_name_list), ('Number of frames and number of GT lines should be equal.')
num_frames = len(gt)
bboxes = np.zeros((num_frames,4)) # stores tracker's output for evaluation
# create an instance of the class Tracker and initialize it with groundtruth
tracker = Tracker(init_bbox, evaluation.start_frame, design, video_folder, frame_name_list)
# used to pad the crops
avg_chan = np.mean(tracker.frame, axis=(0,1))
# fixed multiplicative factors to scale up/down the target
scale_factors = hp.scale_step**np.linspace(-np.ceil(hp.scale_num/2), np.ceil(hp.scale_num/2), hp.scale_num)
scaled_exemplar = scale_factors * tracker.z_sz
min_z = hp.scale_min * tracker.z_sz
max_z = hp.scale_max * tracker.z_sz
min_x = hp.scale_min * tracker.x_sz
max_x = hp.scale_max * tracker.x_sz

TypeError: __init__() takes exactly 3 arguments (6 given)

In [4]:
tracker.frame_padded, tracker.npad = pad_frame(tracker.frame, tracker.bbox.pos, tracker.z_sz, avg_chan);
# template of the target, will be fed to the graph
z_crops = extract_crops(tracker.frame_padded, tracker.npad, tracker.bbox.pos, tracker.z_sz, design.exemplar_sz)

NameError: name 'tracker' is not defined

In [5]:
# cosine window to penalize large displacements
hann_1d = np.expand_dims(np.hanning(design.score_sz*hp.response_up), axis=0)
penalty = np.transpose(hann_1d) * hann_1d
penalty = penalty / np.sum(penalty)

In [6]:
# import pre-trained siamese
net_path=os.path.join(env.root_pretrained,design.net)
X = tf.placeholder(tf.float32)
Z = tf.placeholder(tf.float32)
net = import_siamese(net_path, X, Z)

> Layer 1
		CONV: setting br_conv1f br_conv1b
		CONV: stride 2, filter-group False
		BNORM: setting br_bn1b br_bn1m br_bn1x
		MAX-POOL: size 3 and stride 2
> Layer 2
		CONV: setting br_conv2f br_conv2b
		CONV: stride 1, filter-group True
		BNORM: setting br_bn2b br_bn2m br_bn2x
		MAX-POOL: size 3 and stride 1
> Layer 3
		CONV: setting br_conv3f br_conv3b
		CONV: stride 1, filter-group False
		BNORM: setting br_bn3b br_bn3m br_bn3x
> Layer 4
		CONV: setting br_conv4f br_conv4b
		CONV: stride 1, filter-group True
		BNORM: setting br_bn4b br_bn4m br_bn4x
> Layer 5
		CONV: setting br_conv5f br_conv5b
		CONV: stride 1, filter-group True


In [7]:
# > store initial bbox
for i in xrange(evaluation.start_frame+1, 2):
    print 'Frame '+str(i)+'/'+str(num_frames)
    # call tracker object to read new img
    tracker.get_frame(i, video_folder, frame_name_list)    
    scaled_search_area = tracker.x_sz * scale_factors
    tracker.frame_padded, tracker.npad = pad_frame(tracker.frame, tracker.bbox.pos, scaled_search_area[-1], avg_chan);
    x_crops = extract_crops(tracker.frame_padded, tracker.npad, tracker.bbox.pos, scaled_search_area, design.search_sz)
    with tf.Session() as sess:
#         test_writer = tf.summary.FileWriter('./logs', sess.graph)
        sess.run(tf.global_variables_initializer())
        out = sess.run(net, feed_dict={X:x_crops, Z:z_crops})
    
    if run.visualization:
        with tf.Session() as sess:
            z = sess.run(z_crop)            
            x = sess.run(x_crops)
            
#         z = np.uint8(np.squeeze(z, axis=0))
        z = np.uint8(z[0,:,:,:])
        x0 = np.uint8(x[0,:,:,:])
        x1 = np.uint8(x[1,:,:,:])
        x2 = np.uint8(x[2,:,:,:])
        fig = plt.figure(1)
        ax = fig.add_subplot(111)
        r = patches.Rectangle(
            (tracker.bbox.pos[1]-tracker.bbox.target_sz[1]/2, tracker.bbox.pos[0]-tracker.bbox.target_sz[0]/2),
            tracker.bbox.target_sz[1], tracker.bbox.target_sz[1],
            linewidth=2, edgecolor='r', fill=False)    
        ax.imshow(tracker.frame)
        ax.add_patch(r)
        plt.ion()
        plt.show()
        plt.pause(0.001)

        if run.debug:
            fig_dbg = plt.figure(2)
            ax1 = fig_dbg.add_subplot(141)
            ax2 = fig_dbg.add_subplot(142)
            ax3 = fig_dbg.add_subplot(143)
            ax4 = fig_dbg.add_subplot(144)
            ax1.imshow(z)
            ax2.imshow(x0)
            ax3.imshow(x1)
            ax4.imshow(x2)
            plt.ion()
            plt.show()
            plt.pause(1)

Frame 1/105


TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles.

In [ ]:
plt.close('all')

In [ ]:
# # initialize figure and visualize
# if run.visualization:    
#     fig, ax = plt.subplots(1)
#     r = patches.Rectangle(
#         (tracker.bbox.cx-tracker.bbox.w/2, tracker.bbox.cy-tracker.bbox.h/2),
#         tracker.bbox.w, tracker.bbox.h,
#         linewidth=2, edgecolor='r', fill=False)
#     ax.imshow(tracker.frame)
#     ax.add_patch(r)
#     plt.show()